<a href="https://colab.research.google.com/github/KrishnaKakani-GitHub/Introduction-To-Github/blob/main/Project3_Amazon_Product_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')
# Load the dataset
file_path = '/content/drive/My Drive/MIT CLASS/ratings_Electronics.csv'
df = pd.read_csv(file_path, header=None)
# Display the first few rows of the dataframe
df.head()

In [ ]:
!pip install surprise

In [ ]:
# Importing the necessary libraries and overview of the dataset

import warnings # Used to ignore the warning given as ou
tput of the code
warnings.filterwarnings('ignore')
import numpy as np # Basic libraries of python for numeric
and dataframe computations
import pandas as pd
import matplotlib.pyplot as plt # Basic library for data visualization
import seaborn as sns # Slightly advanced library for data vis
ualization
from collections import defaultdict # A dictionary output that does not rais
e a key error
from sklearn.metrics import mean_squared_error # A performance metrics in sklearn

In [ ]:
# Loading the data

df.columns = ['user_id', 'prod_id', 'rating', 'timestamp'] # Adding column names
df = df.drop('timestamp', axis = 1) # Dropping timestamp
df_copy = df.copy(deep = True) # Copying the data to another DataFrame

In [ ]:
# Get the column containing the users
users = df.user_id
# Create a dictionary from users to their number of ratings
ratings_count = dict()
for user in users:
# If we already have the user, just add 1 to their rating count
if user in ratings_count:
ratings_count[user] += 1
# Otherwise, set their rating count to 1
else:
ratings_count[user] = 1

In [ ]:
RATINGS_CUTOFF = 50
remove_users = []
for user, num_ratings in ratings_count.items():
if num_ratings < RATINGS_CUTOFF:
remove_users.append(user)
df = df.loc[ ~ df.user_id.isin(remove_users)]

In [ ]:
prods = df.prod_id
# Create a dictionary from products to their number of ratings
ratings_count = dict()
for prod in prods:
# If we already have the product, just add 1 to its rating count
if prod in ratings_count:
ratings_count[prod] += 1
# Otherwise, set their rating count to 1
else:
ratings_count[prod] = 1

In [ ]:
# We want our item to have at least 5 ratings to be considered
RATINGS_CUTOFF = 5
remove_users = []
for user, num_ratings in ratings_count.items():
if num_ratings < RATINGS_CUTOFF:
remove_users.append(user)
df_final = df.loc[~ df.prod_id.isin(remove_users)]

In [ ]:
# Print a few rows of the imported dataset
df_final

In [ ]:
# Exploratory Data Analysis

# Check the number of rows and columns and provide observations
rows, columns = df_final.shape
print("No of rows: ", rows)
print("No of columns: ", columns)

In [ ]:
# Check Data types and provide observations
df_final.dtypes

In [ ]:
# Checking For Missing Values

# Check for missing values present and provide observations
df_final.isnull().sum()

In [ ]:
# Summary Statistics

# Summary statistics of 'rating' variable and provide observations
df_final['rating'].describe()

In [ ]:
# Checking the rating distribution

import matplotlib.pyplot as plt
# Create the bar plot and provide observations
plt.figure(figsize=(12, 6))
df_final['rating'].value_counts().plot(kind='bar')
plt.title('Rating Distribution')
plt.xlabel('Ratings')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Checking the number of unique users and items in the dataset

# Number of total rows in the data and number of unique user id and product id in the data
print('The number of observations in the final data = ', len(df_final))
print('Number of unique USERS in Raw data = ', df_final['user_id'].nunique())
print('Number of unique ITEMS in Raw data = ', df_final['prod_id'].nunique())

In [ ]:
# Users with the most number of ratings

# Top 10 users based on the number of ratings
most_rated = df_final.groupby('user_id').size().sort_values(ascending = False)[:10]
most_rated

## Model 1: Rank Based Recommendation System

In [ ]:
df_final.head()

In [ ]:
# Calculate the average rating for each product
average_rating = df_final.groupby('prod_id')['rating'].mean()

# Calculate the count of ratings for each product
count_rating = df_final.groupby('prod_id')['rating'].count()

# Create a DataFrame with calculated average and count of ratings
final_rating = pd.DataFrame({
    'average_rating': average_rating,
    'count_rating': count_rating
})

# Sort the DataFrame by average rating in descending order
final_rating = final_rating.sort_values(by='average_rating', ascending=False)

# Display the first five records of the final_rating DataFrame
final_rating.head()

In [ ]:
def top_n_products(final_rating, n, min_interaction):
    """
    Returns the top N products with the highest average rating,
    considering only products with at least `min_interaction` interactions.
    """
    # Finding products with minimum number of interactions
    recommendations = final_rating.groupby('ProductId').agg({'UserId': 'count'}).reset_index()
    recommendations = recommendations.rename(columns={'UserId': 'Total_Interactions'})

    # Filtering products with at least `min_interaction` interactions
    recommendations = recommendations[recommendations['Total_Interactions'] >= min_interaction]

    # Merging with average ratings
    avg_ratings = final_rating.groupby('ProductId')['Rating'].mean().reset_index()
    avg_ratings = avg_ratings.rename(columns={'Rating': 'Average_Rating'})

    # Joining interactions with average ratings
    recommendations = recommendations.merge(avg_ratings, on='ProductId')

    # Sorting values by average rating (descending)
    recommendations = recommendations.sort_values(by='Average_Rating', ascending=False)

    # Selecting top n products
    top_n_recommendations = recommendations.head(n)

    return top_n_recommendations['ProductId'].tolist()

In [ ]:
def top_n_popular_products(df, n, min_interactions):
    """
    Recommend top N products based on popularity (number of interactions),
    filtering out products with fewer than `min_interactions`.
    """
    # Grouping by ProductId to count total interactions
    recommendations = df.groupby('ProductId').agg({'UserId': 'count'}).reset_index()
    recommendations = recommendations.rename(columns={'UserId': 'Total_Interactions'})

    # Filtering products with at least min_interactions interactions
    recommendations = recommendations[recommendations['Total_Interactions'] >= min_interactions]

    # Sorting values by total interactions in descending order (popularity)
    recommendations = recommendations.sort_values(by='Total_Interactions', ascending=False)

    # Selecting top n products
    top_n_recommendations = recommendations.head(n)

    return top_n_recommendations['ProductId'].tolist()

In [ ]:
def top_n_popular_products(df, n, min_interactions):
    """
    Recommend top N products based on popularity (number of interactions),
    filtering out products with fewer than `min_interactions`.
    """
    # Grouping by ProductId to count total interactions
    recommendations = df.groupby('ProductId').agg({'UserId': 'count'}).reset_index()
    recommendations = recommendations.rename(columns={'UserId': 'Total_Interactions'})

    # Filtering products with at least min_interactions interactions
    recommendations = recommendations[recommendations['Total_Interactions'] >= min_interactions]

    # Sorting values by total interactions in descending order (popularity)
    recommendations = recommendations.sort_values(by='Total_Interactions', ascending=False)

    # Selecting top n products
    top_n_recommendations = recommendations.head(n)

    return top_n_recommendations['ProductId'].tolist()

## Model 2: Collaborative Filtering Recommendation System

In [ ]:
# Building a baseline user-user similarity based recommendation system

# To compute the accuracy of models
from surprise import accuracy
# Class is used to parse a file containing ratings, data should be in structure - user ;
item ; rating
from surprise.reader import Reader
# Class for loading datasets
from surprise.dataset import Dataset
# For tuning model hyperparameters
from surprise.model_selection import GridSearchCV
# For splitting the rating data in train and test datasets
from surprise.model_selection import train_test_split
# For implementing similarity-based recommendation system
from surprise.prediction_algorithms.knns import KNNBasic
# For implementing matrix factorization based recommendation system
from surprise.prediction_algorithms.matrix_factorization import SVD
# for implementing K-Fold cross-validation
from surprise.model_selection import KFold
# For implementing clustering-based recommendation system
from surprise import CoClustering

In [ ]:
def precision_recall_at_k(model, testset, k=10, threshold=3.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user
    user_est_true = defaultdict(list)

    # Making predictions on the test data
    predictions = model.test(testset)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()

    for uid, user_ratings in user_est_true.items():
        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. Set to 0 to avoid division by zero
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. Set to 0 to avoid division by zero
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    # Mean of all the predicted precisions and recalls are calculated
    precision = round((sum(prec for prec in precisions.values()) / len(precisions)), 3)
    recall = round((sum(rec for rec in recalls.values()) / len(recalls)), 3)

    # RMSE for model's predictions
    accuracy.rmse(predictions)

    # Print the metrics
    print('Precision: ', precision)
    print('Recall: ', recall)

    # F1 Score calculation
    if precision + recall > 0:
        f1_score = round((2 * precision * recall) / (precision + recall), 3)
    else:
        f1_score = 0.0
    print('F_1 score: ', f1_score)

In [ ]:
df_final.head()

In [ ]:
# Building the user-user Similarity-based Recommendation System

# Declaring the similarity options
sim_options = {'name': 'cosine',
'user_based': True}
# Initialize the KNNBasic model using sim_options provided, Verbose = False, and setting
random_state = 1
sim_user_user = KNNBasic(sim_options = sim_options, random_state = 1, verbose = False)
# Fit the model on the training data
sim_user_user.fit(trainset)
# Let us compute precision@k, recall@k, and f_1 score using the precision_recall_at_k fun
ction defined above
precision_recall_at_k(sim_user_user)

In [ ]:
# Predicting rating for a sample user with an interacted product
sim_user_user.predict("A3LDPF5FMB782Z", "1400501466", r_ui = 5, verbose = True)

In [ ]:
df_final[df_final.prod_id != "1400501466"].user_id.unique()

In [ ]:
# Predicting rating for a sample user with a non interacted product
sim_user_user.predict("A34BZM6S9L7QI4", "1400501466", verbose = True)

In [ ]:
 # Improving Similarity-based Recommendation System by tuning its hyperparameters

 # Define the Reader with the rating scale
reader = Reader(rating_scale=(1, 5))
# Load the data from the DataFrame
data = Dataset.load_from_df(df[['user_id', 'prod_id', 'rating']], reader)
# Setting up parameter grid to tune the hyperparameters
param_grid = {
'k': [10, 20, 30],
'min_k': [3, 6, 9],
'sim_options': {
'name': ['cosine', 'msd', 'pearson', 'pearson_baseline'],
'user_based': [True]
}
}
# Performing 3-fold cross-validation to tune the hyperparameters
gs = GridSearchCV(KNNBasic, param_grid, measures=['rmse'], cv=3, n_jobs=-1)
# Fitting the data
gs.fit(data)

# Best RMSE score
print(gs.best_score['rmse'])
# Combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

In [ ]:
# Import the necessary classes
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import GridSearchCV, train_test_split
from surprise import accuracy

In [ ]:
!pip install scikit-surprise

In [ ]:
# Performing 3-fold cross-validation to tune the hyperparameters
gs = GridSearchCV(KNNBasic, param_grid, measures=['rmse'], cv=3, n_jobs=-1)

# Fitting the data
gs.fit(data)

# Best parameters and score
print(gs.best_params['rmse'])
print(gs.best_score['rmse'])

# Using the optimal similarity measure for user-user based collaborative filtering
sim_options = {
    'name': gs.best_params['rmse']['sim_options']['name'],
    'user_based': True
}

# Splitting the data into train and test sets
trainset, testset = train_test_split(data, test_size=0.25, random_state=42)

from collections import defaultdict

def precision_recall_at_k(predictions, k=10, threshold=3.5):
    user_est_true = defaultdict(list)

    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()

    for uid, user_ratings in user_est_true.items():
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])
        n_rel_and_rec_k = sum(
            ((true_r >= threshold) and (est >= threshold))
            for (est, true_r) in user_ratings[:k]
        )

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    precision_at_k = sum(prec for prec in precisions.values()) / len(precisions)
    recall_at_k = sum(rec for rec in recalls.values()) / len(recalls)

    return precision_at_k, recall_at_k

In [ ]:
# Create and fit the sim_user_user_optimized model (assuming KNNBasic is available)
from surprise import KNNBasic
sim_user_user_optimized = KNNBasic(sim_options=sim_options)
sim_user_user_optimized.fit(trainset)

# Use sim_user_user_optimized model to recommend for userId "A3LDPF5FMB782Z" and productId 1400501466
# Predict rating for userId="A3LDPF5FMB782Z" and prod_id="1400501466"
user_id = "A3LDPF5FMB782Z"
prod_id = "1400501466"

# Make the prediction
prediction_user_item = sim_user_user_optimized.predict(user_id, prod_id)
print(f"Predicted rating for user {user_id} on product {prod_id}: {prediction_user_item.est:.2f}")

In [ ]:
# Use sim_user_user_optimized model to recommend for userId "A34BZM6S9L7QI4" and productI
d "1400501466"
# Predict rating for userId="A34BZM6S9L7QI4" and prod_id="1400501466"
user_id_new = "A34BZM6S9L7QI4"
# First, check if the user has interacted with the product (not in trainset)
if prod_id not in trainset.ur[trainset.to_inner_uid(user_id_new)]:
# Make the prediction
prediction_new_user_item = sim_user_user_optimized.predict(user_id_new, prod_id)
print(f"Predicted rating for new user {user_id_new} on product {prod_id}: {prediction
_new_user_item.est:.2f}")
else:
print(f"User {user_id_new} has already interacted with product {prod_id}.")

In [ ]:
# 0 is the inner id of the user
sim_user_user_optimized.get_neighbors(0, 5)

In [ ]:
# Implementing  the recommendation algorithm based on optimized KNNBasic model

def get_recommendations(data, user_id, top_n, algo):
    # Creating an empty list to store the recommended product ids
    recommendations = []

    # Creating an user item interactions matrix
    user_item_interactions_matrix = data.pivot(index='user_id', columns='prod_id', values='rating')

    # Extracting those product ids which the user_id has not interacted yet
    non_interacted_products = user_item_interactions_matrix.loc[user_id][
        user_item_interactions_matrix.loc[user_id].isnull()
    ].index.tolist()

    # Looping through each of the product ids which user_id has not interacted yet
    for item_id in non_interacted_products:
        # Predicting the ratings for those non interacted product ids by this user
        est = algo.predict(user_id, item_id).est
        # Appending the predicted ratings
        recommendations.append((item_id, est))

    # Sorting the predicted ratings in descending order
    recommendations.sort(key=lambda x: x[1], reverse=True)

    return recommendations[:top_n]  # Returing top n highest predicted rating products for this user


In [ ]:
# Predicting top 5 products for userId = "A3LDPF5FMB782Z" with similarity based recommendation system

# Making top 5 recommendations for user_id "A3LDPF5FMB782Z" with a similarity-based recommendation engine
recommendations = get_recommendations(df_final, "A3LDPF5FMB782Z", 5, sim_user_user_optimized)

# Building the dataframe for above recommendations with columns "prod_id" and "predicted_ratings"
pd.DataFrame(recommendations, columns=['prod_id', 'predicted_ratings'])


In [ ]:
# Item-Item Similarity-based Collaborative Filtering Recommendation System

from surprise import KNNBasic, Dataset, Reader, accuracy
from surprise.model_selection import train_test_split
import pandas as pd

# Load the dataset (assuming you have already loaded and prepared your dataset)
df = pd.read_csv('/content/drive/My Drive/MIT CLASS/ratings_Electronics.csv', header=None)

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[[0, 1, 2]], reader)

# Define similarity options
sim_options = {
    'name': 'cosine',  # Use cosine similarity
    'user_based': False  # Item-item similarity
}

# Split the data into training and test sets
trainset, testset = train_test_split(data, test_size=0.25, random_state=42)

# Instantiate the KNNBasic model with similarity options
sim_item_item = KNNBasic(sim_options=sim_options, random_state=1, verbose=False)

# Train the model on the training set
sim_item_item.fit(trainset)

# Predict ratings for the test set
predictions = sim_item_item.test(testset)

# Compute precision@k, recall@k, and f1 score with k=10
precision_at_k, recall_at_k = accuracy.precision_recall_at_k(predictions, k=10, threshold=4)
f1_score = 2 * (precision_at_k * recall_at_k) / (precision_at_k + recall_at_k)

print(f'Precision@10: {precision_at_k}')
print(f'Recall@10: {recall_at_k}')
print(f'F1 Score@10: {f1_score}')

In [ ]:
# Assuming sim_item_item is already trained and ready to use

# Specify the user and product IDs for which you want to predict the rating
user_id = 'A3LDPF5FMB782Z'
prod_id = '1400501466'

# Convert user_id and prod_id to inner IDs used by Surprise
user_inner_id = sim_item_item.trainset.to_inner_uid(user_id)
prod_inner_id = sim_item_item.trainset.to_inner_iid(prod_id)

# Predict the rating
rating_prediction = sim_item_item.predict(user_inner_id, prod_inner_id)

# Print the predicted rating
print(f"Predicted rating for user {user_id} on product {prod_id}: {rating_prediction.est:.2f}")


In [ ]:
# Predicting rating for a sample user with a non interacted product
sim_item_item.predict("A34BZM6S9L7QI4", "1400501466", verbose = True)

In [ ]:
# Hyperparameter tuning the item-item similarity-based model

In [ ]:
# Setting up parameter grid to tune the hyperparameters
param_grid = {
    'k': [10, 20, 30],
    'min_k': [3]
}

# Performing 3-fold cross validation to tune the hyperparameters
gs = GridSearchCV(KNNBasic, param_grid, measures=['rmse'], cv=3, n_jobs=-1)

# Fitting the data
gs.fit(data)

# Find the best RMSE score
print(gs.best_score['rmse'])

# Find the combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

In [ ]:
from surprise import KNNBasic, Dataset, Reader, accuracy
from surprise.model_selection import train_test_split

# Load the dataset (assuming you have already loaded and prepared your dataset)
df = pd.read_csv('/content/drive/My Drive/MIT CLASS/ratings_Electronics.csv', header=None)

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[[0, 1, 2]], reader)

# Optimal similarity options for item-item collaborative filtering
sim_options = {
    'name': 'msd',  # Use MSD similarity measure
    'user_based': False  # Item-item similarity
}

# Split the data into training and test sets
trainset, testset = train_test_split(data, test_size=0.25, random_state=42)

# Creating an instance of KNNBasic with optimal hyperparameter values
sim_item_item_optimized = KNNBasic(sim_options=sim_options, random_state=1, verbose=False)

# Training the algorithm on the train set
sim_item_item_optimized.fit(trainset)

# Predictions on the test set
predictions = sim_item_item_optimized.test(testset)

# Compute evaluation metrics: precision@k, recall@k, f1_score, and RMSE
precision_at_k, recall_at_k = accuracy.precision_recall_at_k(predictions, k=10, threshold=4)
f1_score = 2 * (precision_at_k * recall_at_k) / (precision_at_k + recall_at_k)
rmse = accuracy.rmse(predictions)

print(f'Precision@10: {precision_at_k}')
print(f'Recall@10: {recall_at_k}')
print(f'F1 Score: {f1_score}')
print(f'RMSE: {rmse}')

In [ ]:
# Use sim_item_item_optimized model to recommend for userId "A3LDPF5FMB782Z" and productId "1400501466"
# Assuming sim_item_item_optimized is already trained and ready to use

# Specify the user and product IDs for which you want to predict the rating
user_id = 'A3LDPF5FMB782Z'
prod_id = '1400501466'

# Convert user_id and prod_id to inner IDs used by Surprise
user_inner_id = sim_item_item_optimized.trainset.to_inner_uid(user_id)
prod_inner_id = sim_item_item_optimized.trainset.to_inner_iid(prod_id)

# Predict the rating
rating_prediction = sim_item_item_optimized.predict(user_inner_id, prod_inner_id)

# Print the predicted rating
print(f"Predicted rating for user {user_id} on product {prod_id}: {rating_prediction.est:.2f}")

In [ ]:
# Use sim_item_item_optimized model to recommend for userId "A34BZM6S9L7QI4" and productId "1400501466"
# Predict rating for userId="A34BZM6S9L7QI4" and prod_id="1400501466"

user_id_new = "A34BZM6S9L7QI4"
prod_id = "1400501466"

# Convert user_id_new and prod_id to inner IDs used by Surprise
user_inner_id_new = sim_item_item_optimized.trainset.to_inner_uid(user_id_new)
prod_inner_id = sim_item_item_optimized.trainset.to_inner_iid(prod_id)

# First, check if the user has interacted with the product (not in trainset)
if prod_inner_id not in sim_item_item_optimized.trainset.ur[user_inner_id_new]:
    # Make the prediction
    prediction_new_user_item = sim_item_item_optimized.predict(user_inner_id_new, prod_inner_id)
    print(f"Predicted rating for new user {user_id_new} on product {prod_id}: {prediction_new_user_item.est:.2f}")
else:
    print(f"User {user_id_new} has already interacted with product {prod_id}.")


In [ ]:
# Identifying similar items to a given item (nearest neighbors)
sim_item_item_optimized.get_neighbors(0, 5)

In [ ]:
# Making top 5 recommendations for user_id A1A5KUIIIHFF4U with similarity-based recommendation engine.
recommendations = get_recommendations(df_final, "A1A5KUIIIHFF4U", 5, sim_item_item_optimized)


In [ ]:
# Building the dataframe for above recommendations with columns "prod_id" and "predicted_ratings"
pd.DataFrame(recommendations, columns=['prod_id', 'predicted_ratings'])

## Model 3: Matrix Factorization

In [ ]:
# Using SVD matrix factorization. Use random_state = 1
svd = SVD(random_state=1)

# Train the algorithm on the train set
svd.fit(trainset)

# Training the algorithm on the train set
svd.fit(trainset)

# Use the function precision_recall_at_k to compute precision@k, recall@k, F1-Score, and RMSE
precision_at_k, recall_at_k = accuracy.precision_recall_at_k(predictions, k=10, threshold=4)

In [ ]:
# Making prediction
svd.predict("A3LDPF5FMB782Z", "1400501466", r_ui = 5, verbose = True)

In [ ]:
# Making prediction
svd.predict("A34BZM6S9L7QI4", "1400501466", verbose = True)

In [ ]:
# Set the parameter space to tune
param_grid = {'n_epochs': [10, 20, 30], 'lr_all': [0.001, 0.005, 0.01],
              'reg_all': [0.2, 0.4, 0.6]}

# Performing 3-fold gridsearch cross-validation
gs_ = GridSearchCV(, , measures=['rmse'], cv=3, n_jobs=-1)

# Fitting data
gs_.fit(data)

# Best RMSE score
print(gs_.best_score['rmse'])

# Combination of parameters that gave the best RMSE score
print(gs_.best_params['rmse'])

In [ ]:
# Build the optimized SVD model using optimal hyperparameter search. Use random_state = 1
svd_optimized = SVD(random_state=1)

# Train the algorithm on the train set
svd_optimized.fit(trainset)

# Use the function precision_recall_at_k to compute precision@k, recall@k, F1-Score, and RMSE
precision_at_k, recall_at_k = accuracy.precision_recall_at_k(predictions, k=10, threshold=4)

In [ ]:
# Use svd_algo_optimized model to recommend for userId "A3LDPF5FMB782Z" and productId "1400501466"
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split

# Assuming you have loaded and prepared your dataset
# Load the dataset
df = pd.read_csv('/content/drive/My Drive/MIT CLASS/ratings_Electronics.csv', header=None)
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[[0, 1, 2]], reader)

# Split the data into training and test sets
trainset, testset = train_test_split(data, test_size=0.25, random_state=42)

# Creating an instance of SVD with optimal hyperparameter values
svd_algo_optimized = SVD(n_factors=100, n_epochs=20, lr_all=0.005, reg_all=0.02, random_state=42)

# Training the algorithm on the train set
svd_algo_optimized.fit(trainset)

# Predict rating for user A3LDPF5FMB782Z and product 1400501466
user_id_1 = "A3LDPF5FMB782Z"
prod_id_1 = "1400501466"

# Convert user_id and prod_id to inner IDs used by Surprise
user_inner_id_1 = svd_algo_optimized.trainset.to_inner_uid(user_id_1)
prod_inner_id_1 = svd_algo_optimized.trainset.to_inner_iid(prod_id_1)

# Predict the rating
rating_prediction_1 = svd_algo_optimized.predict(user_inner_id_1, prod_inner

In [ ]:
# Use svd_algo_optimized model to recommend for userId "A34BZM6S9L7QI4" and productId "1400501466"
# Predict rating for userId="A34BZM6S9L7QI4" and prod_id="1400501466"
user_id_2 = "A34BZM6S9L7QI4"
prod_id_2 = "1400501466"

# Convert user_id and prod_id to inner IDs used by Surprise
user_inner_id_2 = svd_algo_optimized.trainset.to_inner_uid(user_id_2)
prod_inner_id_2 = svd_algo_optimized.trainset.to_inner_iid(prod_id_2)

# First, check if the user has interacted with the product (not in trainset)
if prod_inner_id_2 not in svd_algo_optimized.trainset.ur[user_inner_id_2]:
    # Make the prediction
    prediction_new_user_item = svd_algo_optimized.predict(user_inner_id_2, prod_inner_id_2)
    print(f"Predicted rating for new user {user_id_2} on product {prod_id_2}: {prediction_new_user_item.est:.2f}")
else:
    print(f"User {user_id_2} has already interacted with product {prod_id_2}.")